# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [3]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [4]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [5]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [6]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [7]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{'_id': {'$oid': '52b213b38594d8a2be17c780'},
  'approvalfy': 1999,
  'board_approval_month': 'November',
  'boardapprovaldate': '2013-11-12T00:00:00Z',
  'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  'closingdate': '2018-07-07T00:00:00Z',
  'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
  'countrycode': 'ET',
  'countryname': 'Federal Democratic Republic of Ethiopia',
  'countryshortname': 'Ethiopia',
  'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  'envassesmentcategorycode': 'C',
  'grantamt': 0,
  'ibrdcommamt': 0,
  'id': 'P129828',
  'idacommamt': 130000000,
  'impagency': 'MINISTRY OF EDUCATION',
  'lendinginstr': 'Investment Project Financing',
  'lendinginstrtype': 'IN',
  'lendprojectcost': 550000000,
  'majorsector_percent': [{'Name': 'Education', 'Percent': 46},
   {'Name': 'Education', 'Percent': 26},
   {'Name': 'Public Administration, Law, and Justice', 'Percent': 16},
   {'Name': 'Educatio

In [8]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [9]:
import pandas as pd
import json

Solution to problem 1: I indexed the column of interest and ran the pandas method value_counts on the result, selecting only the first 10 highest values and printing the result.

In [10]:
json_df = pd.read_json('data/world_bank_projects.json')
most_projects = json_df['countryshortname'].value_counts()[0:10]
print (most_projects)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64


Solution to problem 2: This was a little more complicated because I had to pull the string I needed out of a nested dictionary contained in the dataframe column mjtheme_namecode. I did this by looping over a series of the name codes, pulling out the actual names of each and appending them to a list. Then I intialized a counter object to find the most common occurences in my list and print the results. 

In [11]:
from collections import Counter
json_df = pd.read_json('data/world_bank_projects.json')
project_themes = json_df['mjtheme_namecode']
project_code_names = []
for each in project_themes:
    project_code_names.append(each[0]['name'])
top_ten = Counter(project_code_names)
print(top_ten.most_common(10))

[('Environment and natural resources management', 85), ('Human development', 72), ('Public sector governance', 64), ('Social protection and risk management', 57), ('Rural development', 56), ('Financial and private sector development', 53), ('Social dev/gender/inclusion', 43), ('Trade and integration', 25), ('Urban development', 23), ('Economic management', 11)]


Solution to problem 3: I normalized the JSON strings contained in the nexted dictionaries under 'mjtheme_namecode and made a dictonary out of the codes and names. Three values were left blank so I manually filled them in. Then I loaded a new DF from the JSON file, only keeping the country code and name. Finally I pulled the code column from my normalized table, and mapped the dictionary to it, filling in a final column, theme name. In the original nested dictionary, many projects have more than one theme code associated, but I just kept the first theme code given (presuming this to be the primary classification).

In [120]:
data = json.load(open('data/world_bank_projects.json'))
fill = json_normalize(data,['mjtheme_namecode'])
keys = fill['code']
values = fill['name']
code_dict = dict(zip(keys,values))
print('Dict with missing values...', code_dict)

code_dict['1'] = 'Economic Management'
code_dict['6'] = 'Social Protection and Risk Management'
code_dict['5'] = 'Trade and Integration'
print('')
print('Dict manually filled...', code_dict)

missing_codes_df = pd.read_json('data/world_bank_projects.json')
missing_codes_df = missing_codes_df[['countrycode','countryname']]
missing_codes_df['primary_theme_code'] = fill['code']
missing_codes_df['primary_theme_name'] = missing_codes_df['primary_theme_code'].map(code_dict)
missing_codes_df.head()

Dict with missing values... {'8': 'Human development', '11': 'Environment and natural resources management', '1': '', '6': '', '5': 'Trade and integration', '2': 'Public sector governance', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}

Dict manually filled... {'8': 'Human development', '11': 'Environment and natural resources management', '1': 'Economic Management', '6': 'Social Protection and Risk Management', '5': 'Trade and Integration', '2': 'Public sector governance', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


,countrycode,countryname,primary_theme_code,primary_theme_name
0,ET,Federal Democratic Republic of Ethiopia,8,Human development
1,TN,Republic of Tunisia,11,Environment and natural resources management
2,TV,Tuvalu,1,Economic Management
3,RY,Republic of Yemen,6,Social Protection and Risk Management
4,LS,Kingdom of Lesotho,5,Trade and Integration
